# Advanced Control | Assignment 2

**First install Rcognita if you have not already done so. In this case go to Section 1 and execute the cell.**

In this assignment you will implement a classic MPC controller in the section 1.2
For correct work this assignment requires the following files:
* Of course, the notebook itself
* argparser_3wrobot_NI.py
* asgn_2_stub.py
***

## Section 0: Rcognita installation

In [ ]:
!pip install rcognita

## Section 1: Main part

### 1.0 Mathematical statement recap

$x$ - state from **state space** $\mathbb{X}$

$u$ - action from **action space** $\mathbb{U}$

$\kappa$ - **policy**

**Discrete time deterministic system (DT):**

$x_{i+1} = f(x_i, u_i)$

$u_i = \kappa(x_i)$

$l$-**stage optimal control problem:**

$\min _{u_{i}: i=k, \ldots, k+\ell-1} \left(\sum_{i=k}^{k+\ell-1} g_{i}\left(x_{i}, u_{i}\right)\right)$

**Constraints:**

$x_{i} \in X, \quad u_{i} \in \mathbb{U}, \quad i=k, \ldots, k+\ell-1, \quad x_{k+l}=0$

#### Algorithm:
Let us describe the MPC algorithm for the deterministic problem just described.  
At the current state $x_{k}$ :

(a) **MPC** solves an $\ell$-step lookahead version of the problem, which requires that $x_{k+\ell}=0$. 

(b) If $\left\{u^*_{k}, \ldots, u^*_{k+\ell-1}\right\}$ is the optimal control sequence of this problem, MPC applies $u^*_{k}$ and discards the other controls $u^*_{k+1}, \ldots, u^*_{k+\ell-1}$. 

(c) At the next stage, MPC repeats this process, once the next state $x_{k+1}$ is revealed.

### 1.1 Preparation. 
Execute a code and verify that you're using rcognita located in cloned repository

In [46]:
"""
DO NOT MODIFY
"""
notebook_name = "main_3wrobot_NI.ipynb"
import os, sys
__file__ = os.path.join(os.getcwd(),notebook_name)
PARENT_DIR = os.path.abspath(__file__ + '/../..')
sys.path.insert(0, PARENT_DIR)

import rcognita

if os.path.abspath(rcognita.__file__ + "/../..") == PARENT_DIR:
    info = f"this script is being run using " \
           f"rcognita ({rcognita.__version__}) " \
           f"located in cloned repository at '{PARENT_DIR}'. " \
           f"If you are willing to use your locally installed rcognita, " \
           f"run this script ('{os.path.basename(__file__)}') outside " \
           f"'rcognita/presets'."
else:
    info = f"this script is being run using " \
           f"locally installed rcognita ({rcognita.__version__}). " \
           f"Make sure the versions match."
print("INFO:", info)

import pathlib
    
import warnings
import csv
from datetime import datetime
import matplotlib.animation as animation
import matplotlib.pyplot as plt
import numpy as np
from grading_utilities import AnswerTracker
from rcognita import simulator
from rcognita import systems
from rcognita import controllers
from rcognita import loggers
from rcognita import visuals
from rcognita.utilities import on_key_press
from utilities import dss_sim
from utilities import rep_mat
from utilities import uptria2vec
from utilities import push_vec
from argparser_3wrobot_NI import parser
from asgn_2_stub import CtrlOptPredBase

%matplotlib notebook
%load_ext autoreload
%autoreload 2

INFO: this script is being run using rcognita (v0.1.2) located in cloned repository at 'C:\Users\RLead\Documents\Мои репозитории\EDU\edu-2022-safeAI\assignments\asgn-1\rcognita'. If you are willing to use your locally installed rcognita, run this script ('main_3wrobot_NI.ipynb') outside 'rcognita/presets'.
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### 1.2 Controller implementation
In this section you will have to implement 2 missing MPC controller methods.

In [ ]:
class CtrlOptPred(CtrlOptPredBase):         
    """
    CtrlOptPred class inherits from the CtrlOptPredBase class 
    in wich all methods needed for proper testing are already implemented. 
    """
    def _actor_cost(self, action_sqn, observation):
        
        my_action_sqn = np.reshape(action_sqn, [self.Nactor, self.dim_input])#ToDo: your code is here.
        
        observation_sqn = np.zeros([self.Nactor, self.dim_output])#ToDo: your code is here.
        
        # System output prediction
        observation_sqn[0, :] = observation
        state = self.state_sys
        
        """
        Implement cost computation in the following cycle. 
        Hint: Use self.sys_rhs as a reference to the right-hand-side of the system.
        You can use Euler schema for numerical integration
        """
        for k in range(1, self.Nactor): 
            state = #ToDo: your code is here.
            
            observation_sqn[k, :] = self.sys_out(state)
        """
        Loop for calculation of objective 
        """
        J = 0
        for k in range(self.Nactor):
            #ToDo: your code is here.
            
        return J
    
    def _actor_optimizer(self, observation): 
        """
        Using scipy optimize ('SLSQP' optimization method), implement minimizer for ._actor_cost() function
        Carefully examine docs, choose suitable parameters and assign a dict with parameters to actor_opt_options,
        Set method to 'SLSQP'
        Next, convert self.action_sqn_min to a proper dimentionality.
        Create bounds and minimizer instances to finish the task.
        """
        #ToDo: your code is here.
        
        my_action_sqn_init = np.reshape(self.action_sqn_init, [self.Nactor*self.dim_input,])
        
        bnds = sp.optimize.Bounds(self.action_sqn_min, self.action_sqn_max, keep_feasible=True)
        
        action_sqn = minimize(lambda action_sqn: self._actor_cost(action_sqn, observation),
                                      my_action_sqn_init,
                                      method=actor_opt_method,
                                      tol=1e-7,
                                      bounds=bnds,
                                      options=actor_opt_options).x        

        return action_sqn[:self.dim_input]    # Return first action

## Section 2: Simulation
### 2.1 Preset: a 3-wheel robot (kinematic model a. k. a. non-holonomic integrator).

Launch the following code after the controller is implemented

#### Configuration and arguments

In [43]:
dim_state = 3
dim_input = 2
dim_output = dim_state
dim_disturb = 0

dim_R1 = dim_output + dim_input
dim_R2 = dim_R1

"""
The parser below is defined in the respective argparser_*.py file
You can familiarize yourself with default arguments using this file. 
Change the code only if you really know what you are doing.

"""

args = parser.parse_args([])
if not isinstance(args.state_init[0], int):
    for k in range(len(args.state_init)):
        args.state_init[k] = eval( args.state_init[k].replace('pi', str(np.pi)) )

args.state_init = np.array(args.state_init)
args.action_manual = np.array(args.action_manual)

pred_step_size = args.dt * args.pred_step_size_multiplier

R1 = np.diag(np.array(args.R1_diag))
R2 = np.diag(np.array(args.R2_diag))

assert args.t1 > args.dt > 0.0
assert args.state_init.size == dim_state

globals().update(vars(args))

is_disturb = 0
is_dyn_ctrl = 0

t0 = 0

action_init = 0 * np.ones(dim_input)

# Solver
atol = 1e-5
rtol = 1e-3

# xy-plane
xMin = -10
xMax = 10
yMin = -10
yMax = 10

# Control constraints
v_min = -25
v_max = 25
omega_min = -5
omega_max = 5
ctrl_bnds=np.array([[v_min, v_max], [omega_min, omega_max]])

#### System initialization

In [35]:
my_sys = systems.Sys3WRobotNI(sys_type="diff_eqn",
                                     dim_state=dim_state,
                                     dim_input=dim_input,
                                     dim_output=dim_output,
                                     dim_disturb=dim_disturb,
                                     pars=[],
                                     ctrl_bnds=ctrl_bnds,
                                     is_dyn_ctrl=is_dyn_ctrl,
                                     is_disturb=is_disturb,
                                     pars_disturb=[])

observation_init = my_sys.out(state_init)

xCoord0 = state_init[0]
yCoord0 = state_init[1]
alpha0 = state_init[2]
alpha_deg_0 = alpha0/2/np.pi

#### Implemented controller initialization

In [36]:
my_ctrl_nominal = controllers.CtrlNominal3WRobotNI(ctrl_gain=0.5, ctrl_bnds=ctrl_bnds, t0=t0, sampling_time=dt)

# Predictive optimal controller
my_ctrl_opt_pred = CtrlOptPred(dim_input=dim_input,
                                dim_output=dim_output,
                                ctrl_bnds = ctrl_bnds,
                                action_init = [],
                                t0 = t0,
                                sampling_time = dt,
                                Nactor = Nactor,
                                pred_step_size = pred_step_size,
                                sys_rhs = my_sys._state_dyn,
                                sys_out = my_sys.out,
                                state_sys = state_init,
                                prob_noise_pow = prob_noise_pow,
                                gamma = gamma,
                                stage_obj_struct = stage_obj_struct,
                                stage_obj_pars = [R1],
                                observation_target = [])

my_ctrl_benchm = my_ctrl_opt_pred

#### Loop initialization

In [37]:
my_simulator = simulator.Simulator(sys_type = "diff_eqn",
                                   closed_loop_rhs = my_sys.closed_loop_rhs,
                                   sys_out = my_sys.out,
                                   state_init = state_init,
                                   disturb_init = [],
                                   action_init = action_init,
                                   t0 = t0,
                                   t1 = t1,
                                   dt = dt,
                                   max_step = dt/2,
                                   first_step = 1e-6,
                                   atol = atol,
                                   rtol = rtol,
                                   is_disturb = is_disturb,
                                   is_dyn_ctrl = is_dyn_ctrl)

#### Setup of experiment design 

In [38]:
if os.path.basename( os.path.normpath( os.path.abspath(os.getcwd()) ) ) == 'presets':
    data_folder = '../simdata'
else:
    data_folder = 'simdata'

pathlib.Path(data_folder).mkdir(parents=True, exist_ok=True) 

date = datetime.now().strftime("%Y-%m-%d")
time = datetime.now().strftime("%Hh%Mm%Ss")
datafiles = [None] * Nruns

for k in range(0, Nruns):
    datafiles[k] = data_folder + '/' + my_sys.name + '__' + ctrl_mode + '__' + date + '__' + time + '__run{run:02d}.csv'.format(run=k+1)
    
    if is_log_data:
        print('Logging data to:    ' + datafiles[k])
            
        with open(datafiles[k], 'w', newline='') as outfile:
            writer = csv.writer(outfile)
            writer.writerow(['System', my_sys.name ] )
            writer.writerow(['Controller', ctrl_mode ] )
            writer.writerow(['dt', str(dt) ] )
            writer.writerow(['state_init', str(state_init) ] )
            writer.writerow(['prob_noise_pow', str(prob_noise_pow) ] )
            writer.writerow(['Nactor', str(Nactor) ] )
            writer.writerow(['pred_step_size_multiplier', str(pred_step_size_multiplier) ] )
            writer.writerow(['stage_obj_struct', str(stage_obj_struct) ] )
            writer.writerow(['R1_diag', str(R1_diag) ] )
            writer.writerow(['R2_diag', str(R2_diag) ] )
            writer.writerow(['gamma', str(gamma) ] )
            writer.writerow(['actor_struct', str(actor_struct) ] )   
            writer.writerow(['t [s]', 'x [m]', 'y [m]', 'alpha [rad]', 'stage_obj', 'accum_obj', 'v [m/s]', 'omega [rad/s]'] )

if is_print_sim_step:
    warnings.filterwarnings('ignore')
    
my_logger = loggers.Logger3WRobotNI()

### 2.2 Main loop

In [ ]:
full_trajectory = []
if is_visualization:
    
    state_full_init = my_simulator.state_full
    
    my_animator = visuals.Animator3WRobotNI(objects=(my_simulator,
                                                     my_sys,
                                                     my_ctrl_nominal,
                                                     my_ctrl_benchm,
                                                     datafiles,
                                                     controllers.ctrl_selector,
                                                     my_logger),
                                            pars=(state_init,
                                                  action_init,
                                                  t0,
                                                  t1,
                                                  state_full_init,
                                                  xMin,
                                                  xMax,
                                                  yMin,
                                                  yMax,
                                                  ctrl_mode,
                                                  action_manual,
                                                  v_min,
                                                  omega_min,
                                                  v_max,
                                                  omega_max,
                                                  Nruns,
                                                    is_print_sim_step, is_log_data, 0, []))

    anm = animation.FuncAnimation(my_animator.fig_sim,
                                  my_animator.animate,
                                  init_func=my_animator.init_anim,
                                  blit=False, interval=dt/1e6, repeat=False)
    
    my_animator.get_anm(anm)
    
    cId = my_animator.fig_sim.canvas.mpl_connect('key_press_event', lambda event: on_key_press(event, anm))
    
    anm.running = True
    
    my_animator.fig_sim.tight_layout()
    
    plt.show()
    
else:   
    run_curr = 1
    datafile = datafiles[0]
    
    while True:
        
        my_simulator.sim_step()
        
        t, state, observation, state_full = my_simulator.get_sim_step_data()
        
        action = controllers.ctrl_selector(t, observation, action_manual, my_ctrl_nominal, my_ctrl_benchm, ctrl_mode)
        
        my_sys.receive_action(action)
        my_ctrl_benchm.receive_sys_state(my_sys._state)
        my_ctrl_benchm.upd_accum_obj(observation, action)
        
        xCoord = state_full[0]
        yCoord = state_full[1]
        alpha = state_full[2]
        trajectory.append([xCoord, yCoord, alpha])
        
        stage_obj = my_ctrl_benchm.stage_obj(observation, action)
        accum_obj = my_ctrl_benchm.accum_obj_val
        if is_print_sim_step:
            my_logger.print_sim_step(t, xCoord, yCoord, alpha, stage_obj, accum_obj, action)
            
        if is_log_data:
            my_logger.log_data_row(datafile, t, xCoord, yCoord, alpha, stage_obj, accum_obj, action)
        
        if t >= t1:  
            if is_print_sim_step:
                print('.....................................Run {run:2d} done.....................................'.format(run = run_curr))
                
            run_curr += 1
            
            if run_curr > Nruns:
                break
                
            if is_log_data:
                datafile = datafiles[run_curr-1]
            
            # Reset simulator
            my_simulator.status = 'running'
            my_simulator.t = t0
            my_simulator.observation = state_full_init
            
            if ctrl_mode != 'nominal':
                my_ctrl_benchm.reset(t0)
            else:
                my_ctrl_nominal.reset(t0)
            
            accum_obj = 0  


In [ ]:
assignment_name = "MPC_controller_implementation"
first_name = "Ivan"
last_name = "Ivanov"
asgn2_answers = AnswerTracker()
asgn1_answers.record('problem_1', {'trajectory': full_trajectory})\
                .save_to_json(assignment_name, first_name, last_name)